In [8]:
cd ..

/Users/kangjiwon/Desktop/LG Aimers


In [9]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from phase2_module.src.utils.top_score_instance import check_the_score
from tqdm import tqdm

# 레이블 인코딩할 칼럼들
df_train_data = pd.read_csv("phase2_module/data/train.csv") # 학습용 데이터
df_test_data = pd.read_csv("phase2_module/data/submission.csv") # 테스트 데이터(제출파일의 데이터

drop_columns =['customer_country.1']
df_train_data = df_train_data.drop(columns=drop_columns)
df_test_data=df_test_data.drop(columns=drop_columns)

def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""
    my_dict = {}
    # 모든 요소를 문자열로 변환
    series = series.astype(str)
    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train_data[label_columns], df_test_data[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])
for col in label_columns:  
    df_train_data[col] = df_all.iloc[: len(df_train_data)][col]
    df_test_data[col] = df_all.iloc[len(df_train_data) :][col]
    
stratkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
result = {"f1":[], "precision":[], "recall":[]}
train_result = {"f1":[], "precision":[], "recall":[]}
X = df_train_data.fillna(0).drop(["is_converted"],axis = 1)
Y = df_train_data['is_converted']

model = DecisionTreeClassifier(random_state=42)

# k-fold
for train_idx, test_idx in tqdm(stratkfold.split(X,Y)):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]
    # 모델 훈련
    model.fit(x_train, y_train)
    
    train_pred = model.predict(x_train)
    predict = model.predict(x_test)
    
    score_result = check_the_score(train_pred, y_train)
    for name, score in score_result.items():
        train_result[name].append(score)
        
    score_result = check_the_score(predict, y_test)
    for name, score in score_result.items():
        result[name].append(score)
        
print('----[K-Fold Train Score]-----')
for name, score_list in train_result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')
    
print('----[K-Fold Validation Score]-----')
for name, score_list in result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

5it [00:26,  5.22s/it]

----[K-Fold Train Score]-----
f1 score : 0.9982 / STD: (+/- 0.0001)
precision score : 0.9968 / STD: (+/- 0.0002)
recall score : 0.9996 / STD: (+/- 0.0003)
----[K-Fold Validation Score]-----
f1 score : 0.7790 / STD: (+/- 0.0100)
precision score : 0.7858 / STD: (+/- 0.0101)
recall score : 0.7725 / STD: (+/- 0.0127)


In [10]:
feature_importances = model.feature_importances_

# Get feature names
feature_names = x_train.columns

# Create a dictionary to map feature names to their importance scores
feature_importance_dict = dict(zip(feature_names, feature_importances))

sorted_importance_dict = {k: v for k, v in sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True)}

# Print feature names along with their importance scores after sorting
for feature_name, importance in sorted_importance_dict.items():
    print(f"{feature_name}: {importance}")

customer_idx: 0.5284349080982401
lead_owner: 0.11676529865031672
lead_desc_length: 0.04782433640436715
customer_type: 0.04295820353600521
customer_country: 0.03891264580650572
com_reg_ver_win_rate: 0.03655791155500188
product_category: 0.022758172346068646
response_corporate: 0.020160156828262613
bant_submit: 0.018394058562625284
customer_job: 0.01812934296392605
customer_position: 0.015276883292120326
business_subarea: 0.013214868625233665
historical_existing_cnt: 0.012114080646206244
product_modelname: 0.00977153827076376
product_subcategory: 0.008799407107261221
inquiry_type: 0.00864669847195783
enterprise: 0.0075829013822340564
business_unit: 0.006206096394642415
ver_win_rate_x: 0.00555303710758885
expected_timeline: 0.0050151021855335915
ver_win_ratio_per_bu: 0.004892115944352633
business_area: 0.004771741819409024
id_strategic_ver: 0.00365716281460252
ver_pro: 0.00205120172963358
ver_cus: 0.0011300514420388313
it_strategic_ver: 0.00021103900755106914
idit_strategic_ver: 0.0002110

## 점수 확인
* customer_country.1만 제거
* 중복 제거 안함.

In [11]:
s = df_test_data.drop(['id','is_converted'],axis = 1)
pred = model.predict(s.fillna(0))
df_test_data['is_converted'] = pred
df_test_data.to_csv("submission.csv",index = False)
0.4222704266088214

0.4222704266088214

In [20]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from phase2_module.src.utils.top_score_instance import check_the_score
from tqdm import tqdm
# 레이블 인코딩할 칼럼들
df_train_data = pd.read_csv("phase2_module/data/train.csv") # 학습용 데이터
df_test_data = pd.read_csv("phase2_module/data/submission.csv") # 테스트 데이터(제출파일의 데이터)
drop_columns=['com_reg_ver_win_rate', 'id_strategic_ver', 'idit_strategic_ver', 'it_strategic_ver', 'ver_cus', 'ver_pro', 'ver_win_rate_x','customer_country.1', 'ver_win_ratio_per_bu']
# drop_columns=['customer_country.1']

df_train_data = df_train_data.drop(columns=drop_columns)
df_test_data=df_test_data.drop(columns=drop_columns)
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train_data[label_columns], df_test_data[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])
for col in label_columns:  
    df_train_data[col] = df_all.iloc[: len(df_train_data)][col]
    df_test_data[col] = df_all.iloc[len(df_train_data) :][col]
    
stratkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
result = {"f1":[], "precision":[], "recall":[]}
train_result = {"f1":[], "precision":[], "recall":[]}
X = df_train_data.fillna(0).drop(["is_converted"],axis = 1)
Y = df_train_data['is_converted']

model = DecisionTreeClassifier(random_state=42)

# k-fold
for train_idx, test_idx in tqdm(stratkfold.split(X,Y)):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]
    # 모델 훈련
    model.fit(x_train, y_train)
    
    train_pred = model.predict(x_train)
    predict = model.predict(x_test)
    
    score_result = check_the_score(train_pred, y_train)
    for name, score in score_result.items():
        train_result[name].append(score)
        
    score_result = check_the_score(predict, y_test)
    for name, score in score_result.items():
        result[name].append(score)
        
print('----[K-Fold Train Score]-----')
for name, score_list in train_result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')
    
print('----[K-Fold Validation Score]-----')
for name, score_list in result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

5it [00:16,  3.21s/it]

----[K-Fold Train Score]-----
f1 score : 0.9982 / STD: (+/- 0.0001)
precision score : 0.9968 / STD: (+/- 0.0002)
recall score : 0.9996 / STD: (+/- 0.0003)
----[K-Fold Validation Score]-----
f1 score : 0.7837 / STD: (+/- 0.0071)
precision score : 0.7889 / STD: (+/- 0.0101)
recall score : 0.7787 / STD: (+/- 0.0082)


In [21]:
feature_importances = model.feature_importances_

# Get feature names
feature_names = x_train.columns

# Create a dictionary to map feature names to their importance scores
feature_importance_dict = dict(zip(feature_names, feature_importances))

sorted_importance_dict = {k: v for k, v in sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True)}

# Print feature names along with their importance scores after sorting
for feature_name, importance in sorted_importance_dict.items():
    print(f"{feature_name}: {importance}")

customer_idx: 0.5420492282792219
lead_owner: 0.12957421076979964
lead_desc_length: 0.051995787512886846
customer_country: 0.04166531594002793
customer_type: 0.037832918923516395
bant_submit: 0.02759772340612355
business_subarea: 0.02568629919835567
response_corporate: 0.02105373896597311
product_category: 0.019419654113029237
customer_position: 0.01679706307603553
customer_job: 0.014514290882399482
historical_existing_cnt: 0.013736292425566343
business_area: 0.012490003645075725
product_subcategory: 0.011738092305499447
expected_timeline: 0.007337918724218197
business_unit: 0.0072213442597620576
inquiry_type: 0.006689552653840972
product_modelname: 0.00640886751575654
enterprise: 0.006191697402911627


## 점수 확인 
* 파생 변수 컬럼 제거
* 중복 제거 안함

In [14]:
s = df_test_data.drop(['id','is_converted'],axis = 1)
pred = model.predict(s.fillna(0))
df_test_data['is_converted'] = pred
df_test_data.to_csv("submission.csv",index = False)
0.44951590594744123

0.44951590594744123

In [15]:
def label_rows(row):
    if row['product_modelname'] is np.nan and row['product_subcategory'] is np.nan and row['product_category'] is np.nan:
        return 8
    elif row['product_modelname'] is np.nan and row['product_subcategory'] is np.nan:
        return 7
    elif row['product_modelname'] is np.nan and row['product_category'] is np.nan:
        return 6
    elif row['product_subcategory'] is np.nan and row['product_category'] is np.nan:
        return 5
    elif row['product_modelname'] is np.nan:
        return 4
    elif row['product_subcategory'] is np.nan:
        return 3
    elif row['product_category'] is np.nan:
        return 2
    else:
        return 1

In [16]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from phase2_module.src.utils.top_score_instance import check_the_score
from tqdm import tqdm
# 레이블 인코딩할 칼럼들
df_train_data = pd.read_csv("phase2_module/data/train.csv") # 학습용 데이터
df_test_data = pd.read_csv("phase2_module/data/submission.csv") # 테스트 데이터(제출파일의 데이터)
drop_columns=['com_reg_ver_win_rate', 'id_strategic_ver', 'idit_strategic_ver', 'it_strategic_ver', 'ver_cus', 'ver_pro', 'ver_win_rate_x','customer_country.1', 'ver_win_ratio_per_bu']

df_train_data = df_train_data.drop(columns=drop_columns)
df_test_data=df_test_data.drop(columns=drop_columns)
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train_data[label_columns], df_test_data[label_columns]])
df_all['customer_interest'] = df_all.apply(lambda row: label_rows(row), axis=1)
label_columns.append('customer_interest')
for col in label_columns:
    df_all[col] = label_encoding(df_all[col])
for col in label_columns:  
    df_train_data[col] = df_all.iloc[: len(df_train_data)][col]
    df_test_data[col] = df_all.iloc[len(df_train_data) :][col]
    
stratkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
result = {"f1":[], "precision":[], "recall":[]}
train_result = {"f1":[], "precision":[], "recall":[]}
X = df_train_data.fillna(0).drop(["is_converted"],axis = 1)
Y = df_train_data['is_converted']

model = DecisionTreeClassifier(random_state=42)

# k-fold
for train_idx, test_idx in tqdm(stratkfold.split(X,Y)):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]
    # 모델 훈련
    model.fit(x_train, y_train)
    
    train_pred = model.predict(x_train)
    predict = model.predict(x_test)
    
    score_result = check_the_score(train_pred, y_train)
    for name, score in score_result.items():
        train_result[name].append(score)
        
    score_result = check_the_score(predict, y_test)
    for name, score in score_result.items():
        result[name].append(score)
        
print('----[K-Fold Train Score]-----')
for name, score_list in train_result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')
    
print('----[K-Fold Validation Score]-----')
for name, score_list in result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

5it [00:24,  4.86s/it]

----[K-Fold Train Score]-----
f1 score : 0.9982 / STD: (+/- 0.0001)
precision score : 0.9968 / STD: (+/- 0.0002)
recall score : 0.9996 / STD: (+/- 0.0003)
----[K-Fold Validation Score]-----
f1 score : 0.7847 / STD: (+/- 0.0065)
precision score : 0.7905 / STD: (+/- 0.0077)
recall score : 0.7790 / STD: (+/- 0.0076)


In [17]:
feature_importances = model.feature_importances_

# Get feature names
feature_names = x_train.columns

# Create a dictionary to map feature names to their importance scores
feature_importance_dict = dict(zip(feature_names, feature_importances))

sorted_importance_dict = {k: v for k, v in sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True)}

# Print feature names along with their importance scores after sorting
for feature_name, importance in sorted_importance_dict.items():
    print(f"{feature_name}: {importance}")

customer_idx: 0.5426479921557874
lead_owner: 0.12800054844507605
lead_desc_length: 0.047177182228063275
customer_country: 0.041271783491993674
customer_type: 0.038949949918249425
bant_submit: 0.027810358473246734
business_subarea: 0.025864190394024728
product_category: 0.01960355791229521
response_corporate: 0.01956826069394689
customer_position: 0.01668685200951661
customer_job: 0.01605349020889173
product_subcategory: 0.013846460086039566
historical_existing_cnt: 0.012894582341740655
business_area: 0.011632683531470806
business_unit: 0.009661826487110045
product_modelname: 0.007644534598434496
enterprise: 0.007253161220654581
expected_timeline: 0.006557154026449545
inquiry_type: 0.004662674587390821
customer_interest: 0.0022127571896177075


## 점수 확인
* 내 Customer interest만 추가
* 이것도 제출했다 생각했는데 사실 추가하지 않은 상태로 제출되었음.

In [18]:
s = df_test_data.drop(['id','is_converted'],axis = 1)
pred = model.predict(s.fillna(0))
df_test_data['is_converted'] = pred
df_test_data.to_csv("submission.csv",index = False)
0.44951590594744123

0.44951590594744123

In [19]:
feature_importances = model.feature_importances_

# Get feature names
feature_names = x_train.columns

# Create a dictionary to map feature names to their importance scores
feature_importance_dict = dict(zip(feature_names, feature_importances))

sorted_importance_dict = {k: v for k, v in sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True)}

# Print feature names along with their importance scores after sorting
for feature_name, importance in sorted_importance_dict.items():
    print(f"{feature_name}: {importance}")

customer_idx: 0.5426479921557874
lead_owner: 0.12800054844507605
lead_desc_length: 0.047177182228063275
customer_country: 0.041271783491993674
customer_type: 0.038949949918249425
bant_submit: 0.027810358473246734
business_subarea: 0.025864190394024728
product_category: 0.01960355791229521
response_corporate: 0.01956826069394689
customer_position: 0.01668685200951661
customer_job: 0.01605349020889173
product_subcategory: 0.013846460086039566
historical_existing_cnt: 0.012894582341740655
business_area: 0.011632683531470806
business_unit: 0.009661826487110045
product_modelname: 0.007644534598434496
enterprise: 0.007253161220654581
expected_timeline: 0.006557154026449545
inquiry_type: 0.004662674587390821
customer_interest: 0.0022127571896177075
